In [22]:
from selenium import webdriver as wd
import chromedriver_binary
from bs4 import BeautifulSoup
import uuid
import pandas as pd
import time
import pywhatkit as kit
import keyboard


my_number = "+10000000000"
wd = wd.Chrome()
wd.implicitly_wait(10)
soup =  BeautifulSoup(wd.page_source)

# wd.get("https://www.tesla.com/inventory/new/my?TRIM=LRAWD&arrangeby=plh&zip=98074&range=200")

def get_uuid_id():
    return str(uuid.uuid4())

# wd.get("https://www.tesla.com/inventory/new/my?arrangeby=relevance&zip=98074&range=200")



def search_Tesla(url):

    wd.get(url)
    # zipCode = wd.find_element_by_name("zip")
    # zipCode.click()
    # wd.find_element_by_name("zip").clear()
    # zipCode.send_keys("93501")
    
    soup =  BeautifulSoup(wd.page_source)
    results = soup.find("div",{"class": "results-container"})
    rows =[]
    df_Search_Results = pd.DataFrame(columns= ["CarId","Colour", "Price"])
    
    # for none results the loop cannot ittereate over none
    if results:
        for result in results:
            price_raw = result.find("span", {"class": "result-purchase-price"})
            features = result.find("ul",{"class": "result-regular-features"})

            if price_raw and features:
                price = price_raw.text[1:]
                prices= int(price.replace(',', ''))
                # print(prices)
                colour = features.text[0:15]
                # print(colour)
                car_id = get_uuid_id()
                row = [car_id, colour, prices]
                rows.append(row)

    df = pd.DataFrame.from_records(rows,columns=["CarId","Colour","Price"])
    df_Search_Results = pd.concat([df_Search_Results,df])
    
    return  df_Search_Results

def sendWhatsappMsg(cars_query, my_url):
    my_cars = cars_query[['Colour', 'Price']].copy()
    print(my_cars)
    cars = my_cars.to_string(index = False) + "                                                          " + my_url
    kit.sendwhatmsg_instantly(my_number, cars, 30)
    time.sleep(10)
    keyboard.press_and_release('command+w')
    keyboard.press_and_release('command+w')
    time.sleep(2)
    
    
            
def checkAndNotify():
    cars = search_Tesla("https://www.tesla.com/inventory/used/my?TRIM=PAWD&INTERIOR=PREMIUM_WHITE&VehicleHistory=CLEAN&AUTOPILOT=ENHANCED_AUTOPILOT,AUTOPILOT_FULL_SELF_DRIVING&Year=2022,2021&arrangeby=plh&zip=98075")
    cars_query = cars[(cars.Price<=45000)]
    if (not cars_query.empty):
        print("Found your cars")
        sendWhatsappMsg(cars_query, "https://www.tesla.com/inventory/used/my?TRIM=PAWD&INTERIOR=PREMIUM_WHITE&VehicleHistory=CLEAN&AUTOPILOT=ENHANCED_AUTOPILOT,AUTOPILOT_FULL_SELF_DRIVING&Year=2022,2021&arrangeby=plh&zip=98075")
    else:
        print("No new Cars")
              
# schedule.every(5).hours.at(":00").do(checkAndNotify)
        
while(True):
    
    checkAndNotify()
    # schedule.run_pending()
    time.sleep(18000)
    wd.refresh()
        

Found your cars
            Colour  Price
0  Midnight Silver  43800
1  Pearl White Pai  44200


WebDriverException: Message: disconnected: not connected to DevTools
  (failed to check if window was closed: disconnected: not connected to DevTools)
  (Session info: chrome=115.0.5790.114)
